<a href="https://colab.research.google.com/github/bruno-camara/Medicamentos-Brasil/blob/main/An%C3%A1lise_Medicamentos_Brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Análise de Dados da Base de Medicamentos Brasileiros:**

Base disponível em: https://dados.gov.br/dataset/preco-de-medicamentos-no-brasil-consumidor



##Importando a biblioteca **Pandas** e o **Dataset**

In [1]:
import pandas as pd

In [2]:
med = pd.read_csv('https://raw.githubusercontent.com/bruno-camara/Medicamentos-Brasil/main/TA_PRECO_MEDICAMENTO.csv', low_memory=False)

##**Análise inicial** da base de dados

In [3]:
#Verificando as 5 primeiras linhas
med.head()

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,CÓDIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PREÇO,PF Sem Impostos,PF 0%,PF 12%,PF 17%,PF 17% ALC,"PF 17,5%","PF 17,5% ALC",PF 18%,PF 18% ALC,PF 20%,PMC 0%,PMC 12%,PMC 17%,PMC 17% ALC,"PMC 17,5%","PMC 17,5% ALC",PMC 18%,PMC 18% ALC,PMC 20%,RESTRIÇÃO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANÁLISE RECURSAL,LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS),COMERCIALIZAÇÃO 2020,TARJA
0,SALICILATO DE FENILA;ÁCIDO SALICÍLICO;ÓXIDO DE...,33.379.884/0001-96,LABORATORIO SIMOES LTDA.,520500901178410,057600510011,7896210500354,-,-,TALCO ALÍVIO,TALQUEIRA C/ 100 G,D10A - ANTIACNEICOS TÓPICOS,Similar,Regulado,"5,04","5,65","6,53","6,98","6,08","7,03","6,11","7,08","6,15","7,29","7,58","8,72","9,30","8,41","9,37","8,45","9,43","8,50","9,70",Não,Não,Não,Não,NaN,Negativa,Não,Tarja -(*)
1,DIENOGESTE;VALERATO DE ESTRADIOL,56.990.534/0001-67,SCHERING DO BRASIL QUÍMICA E FARMACÊUTICA LTDA,530917060013104,1002001390025,7896116871497,-,-,CHARMELIE,COM REV A + COM REV B + COM REV C + COM REV D ...,G3A4 - PREPARAÇÕES CONTRACEPTIVAS TRIFÁSICAS,Similar,Regulado,"96,83","108,45","125,32","134,01","116,66","134,95","117,37","135,90","118,09","139,83","145,48","167,40","178,63","161,28","179,84","162,26","181,06","163,25","186,12",Não,Não,Não,Não,NaN,Negativa,Não,Tarja Vermelha(*)
2,HIDROCLOROTIAZIDA;MALEATO DE ENALAPRIL,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525500402113310,1002900020141,7897337705905,-,-,CO-RENITEC,"20 MG + 12,5 MG COM CT BL AL/AL X 30",C9B1 - INIBIDORES DA ECA ASSOCIADOS A ANTI-HIP...,Novo,Regulado,"43,76","43,76","49,73","52,72","52,72","53,04","53,04","53,37","53,37","54,70","60,50","68,75","72,88","72,88","73,32","73,32","73,78","73,78","75,62",Não,Não,Não,Não,NaN,Positiva,Não,Tarja Vermelha
3,HIDROCLOROTIAZIDA;MALEATO DE ENALAPRIL,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525500401117312,1002900020158,7897337705912,-,-,CO-RENITEC,10 MG + 25 MG COM CT BL AL/AL X 30,C9B1 - INIBIDORES DA ECA ASSOCIADOS A ANTI-HIP...,Novo,Regulado,"28,74","28,74","32,66","34,63","34,63","34,84","34,84","35,05","35,05","35,93","39,73","45,15","47,87","47,87","48,16","48,16","48,45","48,45","49,67",Não,Não,Não,Não,NaN,Positiva,Não,Tarja Vermelha
4,MONTELUCASTE,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525502006118219,1002900050014,7897337706575,-,-,SINGULAIR,5 MG COM MAST CT 3 BL AL PLAS INC X 10,R3J2 - ANTIASMÁTICOS/DPOC ANTILEUCOTRIENOS SIS...,Novo,Regulado,"44,67","44,67","50,76","53,82","53,82","54,15","54,15","54,48","54,48","55,84","61,75","70,17","74,40","74,40","74,86","74,86","75,32","75,32","77,20",Não,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha


In [4]:
#Verificando os tipos das colunas
med.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26062 entries, 0 to 26061
Data columns (total 40 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   SUBSTÂNCIA                                             26062 non-null  object
 1   CNPJ                                                   26062 non-null  object
 2   LABORATÓRIO                                            26062 non-null  object
 3   CÓDIGO GGREM                                           26062 non-null  int64 
 4   REGISTRO                                               26062 non-null  object
 5   EAN 1                                                  26062 non-null  object
 6   EAN 2                                                  26062 non-null  object
 7   EAN 3                                                  26062 non-null  object
 8   PRODUTO                                                2

In [5]:
#Descrição simples dos dados numéricos:
med.describe()

,CÓDIGO GGREM
count,2.606200e+04
mean,5.238864e+14
std,1.729204e+13
min,5.001001e+14
25%,5.080181e+14
50%,5.252201e+14
75%,5.343004e+14
max,5.972211e+14


##**Tarefas**

###1) Análise com o **arquivo original**

####1.a) Nome dos campos do arquivo

In [6]:
campos = list(med.columns)
print(campos)

['SUBSTÂNCIA', 'CNPJ', 'LABORATÓRIO', 'CÓDIGO GGREM', 'REGISTRO', 'EAN 1', 'EAN 2', 'EAN 3', 'PRODUTO', 'APRESENTAÇÃO', 'CLASSE TERAPÊUTICA', 'TIPO DE PRODUTO (STATUS DO PRODUTO)', 'REGIME DE PREÇO', 'PF Sem Impostos', 'PF 0%', 'PF 12%', 'PF 17%', 'PF 17% ALC', 'PF 17,5%', 'PF 17,5% ALC', 'PF 18%', 'PF 18% ALC', 'PF 20%', 'PMC 0%', 'PMC 12%', 'PMC 17%', 'PMC 17% ALC', 'PMC 17,5%', 'PMC 17,5% ALC', 'PMC 18%', 'PMC 18% ALC', 'PMC 20%', 'RESTRIÇÃO HOSPITALAR', 'CAP', 'CONFAZ 87', 'ICMS 0%', 'ANÁLISE RECURSAL', 'LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS)', 'COMERCIALIZAÇÃO 2020', 'TARJA']


####1.b) O produto com maior preço de fábrica sem imposto

In [7]:
#Tratando o campo PF Sem Impostos:
med['PF Sem Impostos'] = med['PF Sem Impostos'].str.replace(',', '.')
med['PF Sem Impostos'] = pd.to_numeric(med['PF Sem Impostos'])

In [8]:
max_value = med['PF Sem Impostos'].max()
max_PF = med.loc[med['PF Sem Impostos'] == max_value]

In [9]:
print('O produto com maior preço de fábrica sem imposto é:', max_PF['PRODUTO'].unique()[0])

O produto com maior preço de fábrica sem imposto é: ZOLGENSMA


####1.c) Quais os possíveis tipos de produto

In [10]:
tipo_produto = med['TIPO DE PRODUTO (STATUS DO PRODUTO)'].unique()
print('Existem', len(tipo_produto), 'tipos de produto')
print(tipo_produto)

Existem 11 tipos de produto
['Similar' 'Novo' 'Biológico' 'Específico' 'Genérico' 'Fitoterápico'
 'Produto de Terapia Avançada' 'Biológicos' '-(*)' 'Biológico Novo'
 'Radiofármaco']


In [11]:
#Tratando '-(*)' como tipo não informado
med['TIPO DE PRODUTO (STATUS DO PRODUTO)'] = med['TIPO DE PRODUTO (STATUS DO PRODUTO)'].replace('-(*)', 'Não informado')

In [12]:
print(med['TIPO DE PRODUTO (STATUS DO PRODUTO)'].unique())

['Similar' 'Novo' 'Biológico' 'Específico' 'Genérico' 'Fitoterápico'
 'Produto de Terapia Avançada' 'Biológicos' 'Não informado'
 'Biológico Novo' 'Radiofármaco']


####1.d) Produto e preço do genérico mais barato

In [13]:
genericos = med[med['TIPO DE PRODUTO (STATUS DO PRODUTO)'] == 'Genérico']
gen_baratos = genericos[genericos['PF Sem Impostos'] == genericos['PF Sem Impostos'].min()]
gen_baratos

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,CÓDIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PREÇO,PF Sem Impostos,PF 0%,PF 12%,PF 17%,PF 17% ALC,"PF 17,5%","PF 17,5% ALC",PF 18%,PF 18% ALC,PF 20%,PMC 0%,PMC 12%,PMC 17%,PMC 17% ALC,"PMC 17,5%","PMC 17,5% ALC",PMC 18%,PMC 18% ALC,PMC 20%,RESTRIÇÃO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANÁLISE RECURSAL,LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS),COMERCIALIZAÇÃO 2020,TARJA
12864,ESPIRONOLACTONA,43.640.754/0001-19,FUNDAÇÃO PARA O REMÉDIO POPULAR - FURP,510015010025906,1103901850051,7895005805506,-,-,ESPIRONOLACTONA,25 MG COM CT BL AL PLAS TRANS X 60,C3A1 - AGENTES DIURÉTICOS POUPADORES POTÁSSIO ...,Genérico,Regulado,0.39,"0,39",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Não,Sim,Sim,NaN,Positiva,Não,Tarja Vermelha
12874,ESPIRONOLACTONA,43.640.754/0001-19,FUNDAÇÃO PARA O REMÉDIO POPULAR - FURP,510015010026106,1103901850159,7895005805605,-,-,ESPIRONOLACTONA,25 MG COM CT BL AL PLAS TRANS X 90,C3A1 - AGENTES DIURÉTICOS POUPADORES POTÁSSIO ...,Genérico,Regulado,0.39,"0,39",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Não,Sim,Sim,NaN,Positiva,Não,Tarja Vermelha


Resultou em duas substâncias iguais com registros diferentes

In [14]:
#Nome do produto:
print('O produto genérico mais barato é',gen_baratos['PRODUTO'].unique()[0])

O produto genérico mais barato é ESPIRONOLACTONA


In [15]:
#Preço do produto
print('O preço do produto genérico mais barato é:', gen_baratos['PF Sem Impostos'].unique()[0], 'reais')

O preço do produto genérico mais barato é: 0.39 reais


###2) **Output** de todos os registros com o **preço de fábrica sem impostos superior a ‘100’**, com **comercialização 2020** marcada como **‘Sim’** e que sejam **Tarja Vermelha** em um arquivo chamado “output.csv”

In [16]:
#Verificando os possíveis valores do campo TARJA:
med['TARJA'].unique()

array(['Tarja -(*)', 'Tarja Vermelha(*)', 'Tarja  Vermelha',
       'Tarja Vermelha (*)', 'Tarja Venda Livre',
       'Tarja Venda Livre/Sem Tarja (*)', 'Tarja  Preta', '- (*) ',
       'Tarja Preta(*)'], dtype=object)

In [17]:
#Tratando os valores do campo TARJA:
med['TARJA'] = med['TARJA'].replace('Tarja  Vermelha', 'Tarja Vermelha')
med['TARJA'] = med['TARJA'].replace('Tarja Vermelha(*)', 'Tarja Vermelha')
med['TARJA'] = med['TARJA'].replace('Tarja Vermelha (*)', 'Tarja Vermelha')

In [18]:
#Verificando possíveis valores do campo Comercialização 2020:
med['COMERCIALIZAÇÃO 2020'].unique()

array(['Não', 'Sim'], dtype=object)

In [19]:
sup_100 = med[med['PF Sem Impostos'] > 100]
com_2020 = sup_100[sup_100['COMERCIALIZAÇÃO 2020'] == 'Sim']
output = com_2020[com_2020['TARJA'] == 'Tarja Vermelha']
output

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,CÓDIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PREÇO,PF Sem Impostos,PF 0%,PF 12%,PF 17%,PF 17% ALC,"PF 17,5%","PF 17,5% ALC",PF 18%,PF 18% ALC,PF 20%,PMC 0%,PMC 12%,PMC 17%,PMC 17% ALC,"PMC 17,5%","PMC 17,5% ALC",PMC 18%,PMC 18% ALC,PMC 20%,RESTRIÇÃO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANÁLISE RECURSAL,LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS),COMERCIALIZAÇÃO 2020,TARJA
30,VIRUS DA VARICELA,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525502702157417,1002900240018,7897337703345,-,-,VACINA VARICELA (ATENUADA),1350 UFP PO LIOF SOL INJ CT FA VD INC + FA VD...,J7E2 - VACINA CONTRA VARICELLA,Novo,Regulado,103.04,"103,04","117,09","124,14","124,14","124,90","124,90","125,66","125,66","128,80","142,45","161,87","171,62","171,62","172,67","172,67","173,72","173,72","178,06",Não,Não,Sim,Não,NaN,Positiva,Sim,Tarja Vermelha
31,VIRUS DA VARICELA,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525519020021007,1002900240026,00000000000100,5437456192536,-,VACINA VARICELA (ATENUADA),1350 UFP PO LIOF SOL INJ CT 10 FA VD INC + 10 ...,J7E2 - VACINA CONTRA VARICELLA,Novo,Regulado,1030.41,"1030,41","1170,92","1241,46","1241,46","1248,98","1248,98","1256,60","1256,60","1288,01","1424,48","1618,73","1716,25","1716,25","1726,64","1726,64","1737,18","1737,18","1780,60",Não,Não,Sim,Não,NaN,Positiva,Sim,Tarja Vermelha
39,VÍRUS DA HEPATITE A PURIFICADO INATIVADO,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525513100016103,1002900300061,7897337711449,-,-,VAQTA,"50 U SUS INJ IM CT 10 FA VD INC X 1,0 ML",J7E4 - VACINA PARA HEPATITE,Novo,Regulado,874.01,"874,01","993,19","1053,02","1053,02","1059,41","1059,41","1065,87","1065,87","1092,51","1208,27","1373,03","1455,74","1455,74","1464,57","1464,57","1473,50","1473,50","1510,33",Não,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
45,ACETATO DE CASPOFUNGINA,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525500301155311,1002900330017,7897337707381,-,-,CANCIDAS,50 MG PO LIOF SOL INFUS IV CT FA VD TRANS,J2A - AGENTES SISTÊMICOS PARA INFECÇÕES FÚNGICAS,Novo,Regulado,3054.25,"3054,25","3470,74","3679,82","3679,82","3702,12","3702,12","3724,69","3724,69","3817,81",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
46,ACETATO DE CASPOFUNGINA,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525500302151318,1002900330025,7897337707398,-,-,CANCIDAS,70 MG PO LIOF SOL INFUS IV CT FA VD TRANS,J2A - AGENTES SISTÊMICOS PARA INFECÇÕES FÚNGICAS,Novo,Regulado,3934.50,"3934,50","4471,02","4740,36","4740,36","4769,09","4769,09","4798,17","4798,17","4918,13",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26047,VENETOCLAX,15.800.545/0001-50,ABBVIE FARMACÊUTICA LTDA.,543718100003502,1986000140023,8054083012063,-,-,VENCLEXTA,100 MG COM REV CT FR PLAS OPC X 120,L1X9 - TODOS OS OUTROS ANTINEOPLÁSICOS,Novo,Regulado,28197.09,"31580,75","36494,34","39025,22","33972,39","39297,79","34178,30","39574,19","34386,70","40719,93","42364,45","48748,56","52017,67","46964,84","52368,98","47249,49","52725,02","47537,60","54199,58",Não,Não,Não,Não,NaN,Negativa,Sim,Tarja Vermelha
26048,VENETOCLAX,15.800.545/0001-50,ABBVIE FARMACÊUTICA LTDA.,543718100003602,1986000140031,8054083012056,-,-,VENCLEXTA,100 MG COM REV CX BL AL PLAS TRANS X 14 + 100 ...,L1X9 - TODOS OS OUTROS ANTINEOPLÁSICOS,Novo,Regulado,6085.88,"6816,19","7876,71","8422,96","7332,39","8481,79","7376,83","8541,44","7421,81","8788,73","9143,68","10521,58","11227,17","10136,60","11302,99","10198,03","11379,83","10260,22","11698,09",Não,Não,Não,Não,NaN,Negativa,Sim,Tarja Vermelha
26049,VENETOCLAX,15.800.545/0001-50,ABBVIE FARMACÊUTICA LTDA.,543718100003802,1986000140041,8054083012087,-,-,VENCLEXTA,50 MG COM REV CT BL AL PLAS TRANS X 7,L1X9 - TODOS OS OUTRO

In [20]:
#Salvando o resultado no formato csv
output.to_csv('output.csv')

###3) Lendo o arquivo que output.csv e ordenenando o conteúdo por ordem alfabética do campo produto.

In [21]:
output = pd.read_csv('output.csv')

In [22]:
output_sorted = output.sort_values('PRODUTO')

In [23]:
output_sorted.head()

,Unnamed: 0,SUBSTÂNCIA,CNPJ,LABORATÓRIO,CÓDIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PREÇO,PF Sem Impostos,PF 0%,PF 12%,PF 17%,PF 17% ALC,"PF 17,5%","PF 17,5% ALC",PF 18%,PF 18% ALC,PF 20%,PMC 0%,PMC 12%,PMC 17%,PMC 17% ALC,"PMC 17,5%","PMC 17,5% ALC",PMC 18%,PMC 18% ALC,PMC 20%,RESTRIÇÃO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANÁLISE RECURSAL,LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS),COMERCIALIZAÇÃO 2020,TARJA
3018,20242,ACETATO DE ABIRATERONA,05.035.244/0001-23,SUN FARMACÊUTICA DO BRASIL LTDA,532419070024903,1468200780018,7898272945319,-,-,ABBA,250 MG COM CT FR PLAS PEAD OPC X 120,L2B2 - HORMÔNIOS ANTIANDROGÊNICOS CITOSTÁTICOS,Similar,Regulado,5778.02,"6471,38","7478,25","7996,87","6961,46","8052,72","7003,66","8109,36","7046,36","8344,14","8681,13","9989,33","10659,22","9623,81","10731,21","9682,15","10804,17","9741,18","11106,33",Não,Não,Não,Não,NaN,Negativa,Sim,Tarja Vermelha
3222,21861,ANFOTERICINA B,05.333.542/0001-08,TEVA FARMACÊUTICA LTDA.,537516030006817,1557300340037,7898587900201,-,-,ABELCET,5 MG/ML SUS INJ IV CT FA VD TRANS X 20 ML + AGU,J2A - AGENTES SISTÊMICOS PARA INFECÇÕES FÚNGICAS,Novo,Regulado,2722.37,"2722,37","3093,60","3279,96","3279,96","3299,84","3299,84","3319,96","3319,96","3402,96",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
3567,25998,PACLITAXEL,17.625.281/0001-70,CELGENE BRASIL PRODUTOS FARMACÊUTICOS LTDA.,562417080000007,1961400010016,7898957392063,-,-,ABRAXANE,100 MG PO LIOF SUS INJ CT FA VD TRANS,L1C2 - AGENTES ANTINEOPLÁSICOS TAXANOS,Novo,Regulado,854.85,"854,85","971,42","1029,94","1029,94","1036,18","1036,18","1042,50","1042,50","1068,56",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
1283,7420,CLORIDRATO DE DULOXETINA,33.349.473/0001-58,FARMOQUÍMICA S/A,509017070016504,1039001920074,7898040325602,-,-,ABRETIA,60 MG CAP DURA LIB RETARD CT BL AL PLAS TRANS ...,N6A5 - ANTIDEPRESSIVOS SNRI,Similar,Regulado,199.67,"199,67","226,90","240,57","240,57","242,02","242,02","243,50","243,50","249,59","276,03","313,68","332,57","332,57","334,58","334,58","336,62","336,62","345,04",Não,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
2626,16789,ACEBROFILINA,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,528525712137112,1256801590128,7898148292790,-,-,ACEBROFILINA,5 MG/ ML XPE CX 50 FR PLAS OPC X 120 ML + 50 COP,R5C - EXPECTORANTES,Genérico,Regulado,444.47,"444,47","505,08","535,51","535,51","538,75","538,75","542,04","542,04","555,59",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha


In [24]:
output.columns

Index(['Unnamed: 0', 'SUBSTÂNCIA', 'CNPJ', 'LABORATÓRIO', 'CÓDIGO GGREM',
       'REGISTRO', 'EAN 1', 'EAN 2', 'EAN 3', 'PRODUTO', 'APRESENTAÇÃO',
       'CLASSE TERAPÊUTICA', 'TIPO DE PRODUTO (STATUS DO PRODUTO)',
       'REGIME DE PREÇO', 'PF Sem Impostos', 'PF 0%', 'PF 12%', 'PF 17%',
       'PF 17% ALC', 'PF 17,5%', 'PF 17,5% ALC', 'PF 18%', 'PF 18% ALC',
       'PF 20%', 'PMC 0%', 'PMC 12%', 'PMC 17%', 'PMC 17% ALC', 'PMC 17,5%',
       'PMC 17,5% ALC', 'PMC 18%', 'PMC 18% ALC', 'PMC 20%',
       'RESTRIÇÃO HOSPITALAR', 'CAP', 'CONFAZ 87', 'ICMS 0%',
       'ANÁLISE RECURSAL',
       'LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS)',
       'COMERCIALIZAÇÃO 2020', 'TARJA'],
      dtype='object')

In [25]:
output.drop(columns=['Unnamed: 0'])

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,CÓDIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PREÇO,PF Sem Impostos,PF 0%,PF 12%,PF 17%,PF 17% ALC,"PF 17,5%","PF 17,5% ALC",PF 18%,PF 18% ALC,PF 20%,PMC 0%,PMC 12%,PMC 17%,PMC 17% ALC,"PMC 17,5%","PMC 17,5% ALC",PMC 18%,PMC 18% ALC,PMC 20%,RESTRIÇÃO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANÁLISE RECURSAL,LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS),COMERCIALIZAÇÃO 2020,TARJA
0,VIRUS DA VARICELA,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525502702157417,1002900240018,7897337703345,-,-,VACINA VARICELA (ATENUADA),1350 UFP PO LIOF SOL INJ CT FA VD INC + FA VD...,J7E2 - VACINA CONTRA VARICELLA,Novo,Regulado,103.04,"103,04","117,09","124,14","124,14","124,90","124,90","125,66","125,66","128,80","142,45","161,87","171,62","171,62","172,67","172,67","173,72","173,72","178,06",Não,Não,Sim,Não,NaN,Positiva,Sim,Tarja Vermelha
1,VIRUS DA VARICELA,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525519020021007,1002900240026,00000000000100,5437456192536,-,VACINA VARICELA (ATENUADA),1350 UFP PO LIOF SOL INJ CT 10 FA VD INC + 10 ...,J7E2 - VACINA CONTRA VARICELLA,Novo,Regulado,1030.41,"1030,41","1170,92","1241,46","1241,46","1248,98","1248,98","1256,60","1256,60","1288,01","1424,48","1618,73","1716,25","1716,25","1726,64","1726,64","1737,18","1737,18","1780,60",Não,Não,Sim,Não,NaN,Positiva,Sim,Tarja Vermelha
2,VÍRUS DA HEPATITE A PURIFICADO INATIVADO,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525513100016103,1002900300061,7897337711449,-,-,VAQTA,"50 U SUS INJ IM CT 10 FA VD INC X 1,0 ML",J7E4 - VACINA PARA HEPATITE,Novo,Regulado,874.01,"874,01","993,19","1053,02","1053,02","1059,41","1059,41","1065,87","1065,87","1092,51","1208,27","1373,03","1455,74","1455,74","1464,57","1464,57","1473,50","1473,50","1510,33",Não,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
3,ACETATO DE CASPOFUNGINA,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525500301155311,1002900330017,7897337707381,-,-,CANCIDAS,50 MG PO LIOF SOL INFUS IV CT FA VD TRANS,J2A - AGENTES SISTÊMICOS PARA INFECÇÕES FÚNGICAS,Novo,Regulado,3054.25,"3054,25","3470,74","3679,82","3679,82","3702,12","3702,12","3724,69","3724,69","3817,81",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
4,ACETATO DE CASPOFUNGINA,45.987.013/0001-34,ORGANON FARMACÊUTICA LTDA.,525500302151318,1002900330025,7897337707398,-,-,CANCIDAS,70 MG PO LIOF SOL INFUS IV CT FA VD TRANS,J2A - AGENTES SISTÊMICOS PARA INFECÇÕES FÚNGICAS,Novo,Regulado,3934.50,"3934,50","4471,02","4740,36","4740,36","4769,09","4769,09","4798,17","4798,17","4918,13",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Não,Não,Não,NaN,Positiva,Sim,Tarja Vermelha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3600,VENETOCLAX,15.800.545/0001-50,ABBVIE FARMACÊUTICA LTDA.,543718100003502,1986000140023,8054083012063,-,-,VENCLEXTA,100 MG COM REV CT FR PLAS OPC X 120,L1X9 - TODOS OS OUTROS ANTINEOPLÁSICOS,Novo,Regulado,28197.09,"31580,75","36494,34","39025,22","33972,39","39297,79","34178,30","39574,19","34386,70","40719,93","42364,45","48748,56","52017,67","46964,84","52368,98","47249,49","52725,02","47537,60","54199,58",Não,Não,Não,Não,NaN,Negativa,Sim,Tarja Vermelha
3601,VENETOCLAX,15.800.545/0001-50,ABBVIE FARMACÊUTICA LTDA.,543718100003602,1986000140031,8054083012056,-,-,VENCLEXTA,100 MG COM REV CX BL AL PLAS TRANS X 14 + 100 ...,L1X9 - TODOS OS OUTROS ANTINEOPLÁSICOS,Novo,Regulado,6085.88,"6816,19","7876,71","8422,96","7332,39","8481,79","7376,83","8541,44","7421,81","8788,73","9143,68","10521,58","11227,17","10136,60","11302,99","10198,03","11379,83","10260,22","11698,09",Não,Não,Não,Não,NaN,Negativa,Sim,Tarja Vermelha
3602,VENETOCLAX,15.800.545/0001-50,ABBVIE FARMACÊUTICA LTDA.,543718100003802,1986000140041,8054083012087,-,-,VENCLEXTA,50 MG COM REV CT BL AL PLAS TRANS X 7,L1X9 - TODOS OS OUTROS ANTINE

In [26]:
#Salvando o otput.csv ordenado
output_sorted.to_csv('output.csv')

###4) Com o output ordenado:

####4.a) Soma dos preços de fábrica sem imposto? 

In [27]:
print('A soma dos preços de todos os produtos é', round(sum(output_sorted['PF Sem Impostos']), 2), 'reais')

A soma dos preços de todos os produtos é 7184334.67 reais


####4.b) Número de produtos existentes

In [28]:
print('O número de produtos diferentes existentes é:', len(output_sorted['PRODUTO'].unique()))

O número de produtos diferentes existentes é: 1772


####4.c) Existe algum produto que possua a mesma “substância” de um outro produto do arquivo original com preço de fábrica sem impostos inferior a ‘100’?

In [29]:
inf_100 = med[med['PF Sem Impostos'] < 100]
inf_100['SUBSTÂNCIA'].describe()

count              17054
unique              1642
top       COLECALCIFEROL
freq                 312
Name: SUBSTÂNCIA, dtype: object

In [30]:
output_sorted['SUBSTÂNCIA'].describe()

count                        3605
unique                        955
top       OXALATO DE ESCITALOPRAM
freq                           67
Name: SUBSTÂNCIA, dtype: object

In [31]:
inf_100['SUBSTÂNCIA'].isin(output_sorted['SUBSTÂNCIA']).value_counts()

True     10099
False     6955
Name: SUBSTÂNCIA, dtype: int64

Com essa informações descobrimos que existem **produtos com a mesma substância mas com diferentes preços de fábrica**. <br><br>
Mais especificamente, há produtos com preço de fábrica menor que 100 reais com a mesa substância de produtos com o preço de fábrica maior que 100 reais.<br>
Método:


*   Criei um dataset com produtos com preço de fábrica menor que 100 reais.
*   Output_sorted tem apenas produtos com preço de fábrica maior que 100.
*   Usei o método **isin** dos dataframes pandas para verificar se havia **intersecções** dos dois datasets no campo **SUBSTÂNCIA**.
*   Como cheguei em **True diferente de zero** isso significa que houveram intersecções.


